### 0. import libraries and data

In [11]:
# Importation of libraries
import numpy as np
import csv
import pandas as pd

In [12]:
# if < 0 -> = 0, this function is used in the all code
f=lambda a: (abs(a)+a)/2

In [13]:
# Importation of databases
poke_data = pd.read_excel("pokemon_stat.xlsx")
poke_cluster = pd.read_csv("pokemon_cluster.csv", delimiter = ',')
poke_abilities = pd.read_excel("pokemon_abilities.xlsx")
result = pd.read_csv("result_abilities.csv", delimiter = ',')

In [14]:
poke_data = pd.merge(poke_data, poke_cluster, how='left', on="name_poke")
poke_data = pd.merge(poke_data, poke_abilities, how='left', on="name_poke")
poke_data.drop(columns=['Unnamed: 0'], inplace = True)

In [15]:
poke_data['p_resi'] = f(poke_data['def']-10)
poke_data['s_resi'] = f(poke_data['spedef']-10)

### 1. Adding the win_rate

In [16]:
# In the database result, we had a column with the winner of the matchup Poke1 vs Poke2
result.loc[result['Nb_victory_poke1'] == result['Nb_victory_poke2'], 'Winner'] = 'draw'
result.loc[result['Nb_victory_poke1'] > result['Nb_victory_poke2'], 'Winner'] = result['Pokemon1']
result.loc[result['Nb_victory_poke1'] < result['Nb_victory_poke2'], 'Winner'] = result['Pokemon2']

In [17]:
# We create a new dataFrame which contains the win_rate of each Pokemon
win_rate = {
    'name_poke' : [],
    'win_rate' : []
    }

df_win_rate = pd.DataFrame(win_rate)

for i in range(len(poke_data)):
    a = poke_data["name_poke"][i]
    b = ((result[result["Winner"]==poke_data["name_poke"][i]].count()["Pokemon1"])/(len(poke_data)-1)).round(2)
    df_win_rate.loc[i] = [a,b]

df_win_rate

,name_poke,win_rate
0,skarmory,0.53
1,vespiquen,0.34
2,farfetch'd,0.05
3,dodrio,0.40
4,staraptor,0.71
...,...,...
236,alolan ninetales,0.25
237,galarian weezing,0.23
238,togekiss,0.89
239,altaria,0.41


In [18]:
poke_data = pd.merge(poke_data, df_win_rate, how='left', on="name_poke")

In [19]:
poke_data

,name_poke,type1,type2,hp,atk,def,speatk,spedef,speed,Cluster,truant,fulgurance,bulldozer,martial_arts_master,impenetrable,sniper_technician,assassin,p_resi,s_resi,win_rate
0,skarmory,flying,steel,40,16,18,7,11,17,2,False,False,False,False,False,False,False,8.0,1.0,0.53
1,vespiquen,flying,bug,42,14,16,14,16,8,8,False,False,False,False,False,False,False,6.0,6.0,0.34
2,farfetch'd,normal,flying,30,15,10,13,11,14,5,False,False,False,False,False,False,False,0.0,1.0,0.05
3,dodrio,normal,flying,35,18,12,11,11,18,3,False,False,False,False,False,False,False,2.0,1.0,0.40
4,staraptor,normal,flying,50,19,12,12,12,16,3,False,False,False,False,False,False,False,2.0,2.0,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,alolan ninetales,ice,fairy,42,12,10,14,16,18,1,False,False,False,False,False,False,False,0.0,6.0,0.25
237,galarian weezing,poison,fairy,38,15,19,14,12,11,4,False,False,False,False,False,False,False,9.0,2.0,0.23
238,togekiss,flying,fairy,50,11,16,19,18,14,8,False,False,False,False,False,False,False,6.0,8.0,0.89
239,altaria,flying,dragon,45,12,15,12,17,14,8,False,False,False,False,False,False,False,5.0,7.0,0.41


In [20]:
poke_data.to_csv('poke_data.csv')